In [1]:
import pandas as pd
import json
import os
import re
import xml.etree.ElementTree as ET

In [2]:

# 创建一个空的结果DataFrame
# json_file_path = "/Users/fortsun/Desktop/git/restfuzz-sast/output/10-24-OWASPbenchmark/Analyze_result-0-500-不使用taintsanitize.json"
result_df = pd.DataFrame(columns=["entryMethod", "hasDataFlow", "hasCallStack"])

json_file_path = f"/Users/fortsun/Desktop/git/restfuzz-sast/output/server/benchmark2700/Analyze_result.json"
# result_df = pd.read_csv("result500.csv")

# print(result_df.loc[4,:])

with open(json_file_path, "r") as json_file:
    data = json.load(json_file)

for vul_type in data:
    results = data.get(vul_type, [])
    for item in results:
        entry_method = item.get("entryMethod", "")
        # if "dataFlowResult" not in item.keys():
        #     print("?")
        has_data_flow = item.get("dataFlowResult", {}).get("hasDataFlow", False)
        has_data_flow_str = "True" if has_data_flow else "False"
        # result_df = result_df.append({"entryMethod": entry_method, "hasDataFlow": has_data_flow_str}, ignore_index=True)
        result_df.loc[len(result_df)] = pd.Series({"entryMethod": entry_method, "hasDataFlow": has_data_flow_str, "hasCallStack": "True"})

# 输出结果DataFrame
print(result_df)

# 可以选择将结果DataFrame保存为CSV文件
# result_df.to_csv('result2000.csv', index=False)

# print("CSV文件已生成")

                                            entryMethod hasDataFlow  \
0     <org.owasp.benchmark.testcode.BenchmarkTest023...        True   
1     <org.owasp.benchmark.testcode.BenchmarkTest006...        True   
2     <org.owasp.benchmark.testcode.BenchmarkTest001...        True   
3     <org.owasp.benchmark.testcode.BenchmarkTest014...        True   
4     <org.owasp.benchmark.testcode.BenchmarkTest011...        True   
...                                                 ...         ...   
5642  <org.owasp.benchmark.testcode.BenchmarkTest010...        True   
5643  <org.owasp.benchmark.testcode.BenchmarkTest005...        True   
5644  <org.owasp.benchmark.testcode.BenchmarkTest020...        True   
5645  <org.owasp.benchmark.testcode.BenchmarkTest022...        True   
5646  <org.owasp.benchmark.testcode.BenchmarkTest013...        True   

     hasCallStack  
0            True  
1            True  
2            True  
3            True  
4            True  
...           ...  
5642   

In [87]:
# result_df = pd.read_csv("result2000.csv")

In [3]:
def parse_test_id(input_string):
    class_name_match = re.search(r'BenchmarkTest(\d+)', input_string)
    class_number = None
    if class_name_match:
        class_name = class_name_match.group(0)
        class_number = class_name_match.group(1)
        # print("Class Name:", class_name)
        # print("Number:", class_number)
    else:
        print("Pattern not found")
    return class_number
        
parse_test_id(result_df.iloc[0,0])

'02300'

In [4]:
result_df['test-id'] = result_df['entryMethod'].apply(parse_test_id)

Pattern not found


In [5]:
# 根据'test-id'字段合并重复的行，对'hasDataFlow'字段进行合并
# 自定义合并函数
def custom_merge(series):
    for result in series:
        if str(result) == "True":
            return "True"
    return "False"
result_df = result_df.groupby('test-id').agg({"entryMethod": '; '.join, 'hasDataFlow': custom_merge, "hasCallStack": custom_merge}).reset_index()
result_df.loc[4,:]

test-id                                                     00005
entryMethod     <org.owasp.benchmark.testcode.BenchmarkTest000...
hasDataFlow                                                  True
hasCallStack                                                 True
Name: 4, dtype: object

In [6]:
result_df

,test-id,entryMethod,hasDataFlow,hasCallStack
0,00001,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True
1,00002,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True
2,00003,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True
3,00004,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True
4,00005,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True
...,...,...,...,...
2443,02736,<org.owasp.benchmark.testcode.BenchmarkTest027...,True,True
2444,02737,<org.owasp.benchmark.testcode.BenchmarkTest027...,True,True
2445,02738,<org.owasp.benchmark.testcode.BenchmarkTest027...,True,True
2446,02739,<org.owasp.benchmark.testcode.BenchmarkTest027...,True,True


In [7]:

result_df[result_df["hasDataFlow"] == "True"]

,test-id,entryMethod,hasDataFlow,hasCallStack
0,00001,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True
1,00002,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True
2,00003,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True
3,00004,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True
4,00005,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True
...,...,...,...,...
2443,02736,<org.owasp.benchmark.testcode.BenchmarkTest027...,True,True
2444,02737,<org.owasp.benchmark.testcode.BenchmarkTest027...,True,True
2445,02738,<org.owasp.benchmark.testcode.BenchmarkTest027...,True,True
2446,02739,<org.owasp.benchmark.testcode.BenchmarkTest027...,True,True


In [8]:
# 目录中包含XML文件的路径
xml_dir = '/Users/fortsun/Desktop/git/BenchmarkJava/src/main/java/org/owasp/benchmark/testcode'  # 替换为实际的目录路径

# 创建一个空的DataFrame来存储XML数据
xml_data = pd.DataFrame(columns=['test-number', 'benchmark-version', 'category', 'vulnerability', 'cwe'])


# 遍历目录下的XML文件
for filename in os.listdir(xml_dir):
    if filename.endswith('.xml'):
        xml_file = os.path.join(xml_dir, filename)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        # 提取XML数据
        test_number = root.find('test-number').text
        benchmark_version = root.find('benchmark-version').text
        category = root.find('category').text
        vulnerability = root.find('vulnerability').text
        cwe = root.find('cwe').text
        
        # 将提取的数据添加到xml_data DataFrame
        xml_data.loc[len(xml_data)] = pd.Series({'test-number': test_number, 'benchmark-version': benchmark_version,
                                    'category': category, 'vulnerability': vulnerability, 'cwe': cwe})



In [9]:
# 合并标签和实验结果-----------------------------------------------------------------------------------------------
xml_data['id'] = xml_data['test-number'].astype(int)
result_df['id'] = result_df['test-id'].astype(int)

merged_df = result_df.merge(xml_data, how='outer', on='id')
merged_df = merged_df.fillna({'hasDataFlow': "False", "hasCallStack": "False"})
# 使用 fillna() 方法将 'test-id' 列的空白值填充为 'test-number' 列的对应值
merged_df['test-id'].fillna(merged_df['test-number'], inplace=True)

In [12]:
merged_df[merged_df["hasCallStack"] == "True"]

,test-id,entryMethod,hasDataFlow,hasCallStack,id,test-number,benchmark-version,category,vulnerability,cwe
0,00001,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,1,00001,1.2,pathtraver,true,22
1,00002,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,2,00002,1.2,pathtraver,true,22
2,00003,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,3,00003,1.2,hash,true,328
3,00004,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,4,00004,1.2,trustbound,true,501
4,00005,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,5,00005,1.2,crypto,true,327
...,...,...,...,...,...,...,...,...,...,...
2443,02736,<org.owasp.benchmark.testcode.BenchmarkTest027...,True,True,2736,02736,1.2,sqli,false,89
2444,02737,<org.owasp.benchmark.testcode.BenchmarkTest027...,True,True,2737,02737,1.2,sqli,false,89
2445,02738,<org.owasp.benchmark.testcode.BenchmarkTest027...,True,True,2738,02738,1.2,sqli,false,89
2446,02739,<org.owasp.benchmark.testcode.BenchmarkTest027...,True,True,2739,02739,1.2,sqli,false,89


In [11]:
# 条件过滤
alert_vul_tag = "hasDataFlow"
# alert_vul_tag = "hasCallStack"
tp = len(merged_df[(merged_df[alert_vul_tag] == "True") & (merged_df['vulnerability'] == "true")])
fp = len(merged_df[(merged_df[alert_vul_tag] == "True") & (merged_df['vulnerability'] == "false")])
tn = len(merged_df[(merged_df[alert_vul_tag] == "False") & (merged_df['vulnerability'] == "false")])
fn = len(merged_df[(merged_df[alert_vul_tag] == "False") & (merged_df['vulnerability'] == "true")])

# 输出统计结果
print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

tpr = tp/(tp + fn)
fpr = fp/(tn + fp)
youden = tpr - fpr
print("Youden指数： " + str(youden))


True Positives (TP): 1260
False Positives (FP): 1188
True Negatives (TN): 137
False Negatives (FN): 155
Youden指数： -0.006144409627308445


In [98]:
merged_df[merged_df['test-id'].isnull()]

,test-id,entryMethod,hasDataFlow,hasCallStack,id,test-number,benchmark-version,category,vulnerability,cwe


In [99]:

merged_df['id'] = merged_df['test-id'].astype(int)
filtered_df = merged_df[(merged_df['id'] >= 1) & (merged_df['id'].astype(int) <= num)]
filtered_df.to_csv('result-filtered-1500-with-tag.csv', index=False)
print(f"前{num}个benchmark数据点：")
filtered_df


前2000个benchmark数据点：


,test-id,entryMethod,hasDataFlow,hasCallStack,id,test-number,benchmark-version,category,vulnerability,cwe
0,1,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,1,00001,1.2,pathtraver,true,22
1,2,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,2,00002,1.2,pathtraver,true,22
2,3,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,3,00003,1.2,hash,true,328
3,4,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,4,00004,1.2,trustbound,true,501
4,5,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,5,00005,1.2,crypto,true,327
...,...,...,...,...,...,...,...,...,...,...
2399,01514,NaN,False,False,1514,01514,1.2,xss,false,79
2500,01669,NaN,False,False,1669,01669,1.2,xss,false,79
2510,01668,NaN,False,False,1668,01668,1.2,xss,false,79
2612,01592,NaN,False,False,1592,01592,1.2,xss,true,79


In [100]:
# 条件过滤
alert_vul_tag = "hasDataFlow"
# alert_vul_tag = "hasCallStack"
tp = len(filtered_df[(filtered_df[alert_vul_tag] == "True") & (filtered_df['vulnerability'] == "true")])
fp = len(filtered_df[(filtered_df[alert_vul_tag] == "True") & (filtered_df['vulnerability'] == "false")])
tn = len(filtered_df[(filtered_df[alert_vul_tag] == "False") & (filtered_df['vulnerability'] == "false")])
fn = len(filtered_df[(filtered_df[alert_vul_tag] == "False") & (filtered_df['vulnerability'] == "true")])

print(f"前{num}个benchmark数据点：")
# 输出统计结果
print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

tpr = tp/(tp + fn)
fpr = fp/(tn + fp)
youden = tpr - fpr
print("Youden指数： " + str(youden))


前2000个benchmark数据点：
True Positives (TP): 724
False Positives (FP): 518
True Negatives (TN): 444
False Negatives (FN): 314
Youden指数： 0.15903364458277758


In [101]:
filtered_df

,test-id,entryMethod,hasDataFlow,hasCallStack,id,test-number,benchmark-version,category,vulnerability,cwe
0,1,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,1,00001,1.2,pathtraver,true,22
1,2,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,2,00002,1.2,pathtraver,true,22
2,3,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,3,00003,1.2,hash,true,328
3,4,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,4,00004,1.2,trustbound,true,501
4,5,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,5,00005,1.2,crypto,true,327
...,...,...,...,...,...,...,...,...,...,...
2399,01514,NaN,False,False,1514,01514,1.2,xss,false,79
2500,01669,NaN,False,False,1669,01669,1.2,xss,false,79
2510,01668,NaN,False,False,1668,01668,1.2,xss,false,79
2612,01592,NaN,False,False,1592,01592,1.2,xss,true,79


In [102]:
merged_df

,test-id,entryMethod,hasDataFlow,hasCallStack,id,test-number,benchmark-version,category,vulnerability,cwe
0,1,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,1,00001,1.2,pathtraver,true,22
1,2,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,2,00002,1.2,pathtraver,true,22
2,3,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,3,00003,1.2,hash,true,328
3,4,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,4,00004,1.2,trustbound,true,501
4,5,<org.owasp.benchmark.testcode.BenchmarkTest000...,True,True,5,00005,1.2,crypto,true,327
...,...,...,...,...,...,...,...,...,...,...
2735,02249,NaN,False,False,2249,02249,1.2,cmdi,true,78
2736,02513,NaN,False,False,2513,02513,1.2,cmdi,false,78
2737,02275,NaN,False,False,2275,02275,1.2,sqli,true,89
2738,02261,NaN,False,False,2261,02261,1.2,trustbound,true,501


In [103]:
merged_df[merged_df['test-id'] == '00011']

,test-id,entryMethod,hasDataFlow,hasCallStack,id,test-number,benchmark-version,category,vulnerability,cwe


In [104]:
df_fn = filtered_df[(filtered_df[alert_vul_tag] == "False") & (filtered_df['vulnerability'] == "true")]
df_fn

,test-id,entryMethod,hasDataFlow,hasCallStack,id,test-number,benchmark-version,category,vulnerability,cwe
6,7,<org.owasp.benchmark.testcode.BenchmarkTest000...,False,True,7,00007,1.2,cmdi,true,78
10,11,<org.owasp.benchmark.testcode.BenchmarkTest000...,False,True,11,00011,1.2,pathtraver,true,22
12,13,<org.owasp.benchmark.testcode.BenchmarkTest000...,False,True,13,00013,1.2,xss,true,79
13,14,<org.owasp.benchmark.testcode.BenchmarkTest000...,False,True,14,00014,1.2,xss,true,79
22,23,<org.owasp.benchmark.testcode.BenchmarkTest000...,False,True,23,00023,1.2,weakrand,true,330
...,...,...,...,...,...,...,...,...,...,...
1987,02000,<org.owasp.benchmark.testcode.BenchmarkTest020...,False,True,2000,02000,1.2,weakrand,true,330
2243,01667,NaN,False,False,1667,01667,1.2,xss,true,79
2313,01927,NaN,False,False,1927,01927,1.2,xss,true,79
2359,01926,NaN,False,False,1926,01926,1.2,xss,true,79


In [105]:
filtered_df.to_csv(f"./test_result_{num}.csv")